In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill

kill: (30005)kill: (30011)

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)
print(x_train.shape, x_test.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3)


In [5]:
import p8_util_config
p8_util_config.ADANET_MAX_ITERATION_STEPS

6

In [6]:
import p8_util_config
import NNGenerator

nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)

nClasses = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']

estimator = adanet.Estimator(
    head=p8_util_config.dict_adanet_config['adanet_tf_head'],
    
    subnetwork_generator=NNGenerator.NNGenerator(p8_util_config.dict_adanet_config),
            
    adanet_lambda=p8_util_config.dict_adanet_config['adanet_lambda'],#0.0,#.015,
    
    max_iteration_steps=p8_util_config.ADANET_MAX_ITERATION_STEPS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , p8_util_config.NUM_EPOCHS
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        steps=None),
    config=  my_config)

*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/adanet/CNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5abfa96470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
import p8_util_config
print("Global steps= {}".format(p8_util_config.TRAIN_STEPS))
print("MAX Adanet steps= {}".format(p8_util_config.ADANET_MAX_ITERATION_STEPS))

Global steps= 36
MAX Adanet steps= 6


In [8]:
import time
import tensorflow as tf

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train"
                           , x_train
                           , y_train, p8_util_config.NUM_EPOCHS \
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test"
                                   , x_test
                                   , y_test
                                   , p8_util_config.NUM_EPOCHS\
                                   , tuple_dimension=tuple_dimension\
                                   , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0
INFO:tensorflow:Calling model_fn.

*** NNGenerator : layers= (1,2)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork '1_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_0/step/read:0", shape=(), dtype=int64)
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_0/step/read:0", shape=(), dtype=int64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHo

INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/dense_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/dense_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/logits/mixture_weight; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/candidate_t0_2_layer_CNN/adanet_loss; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/candidate_t0_2_layer_CNN/adanet/iteration_0/candidate_t0_2_layer_CNN/adanet_loss/bias

INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_0/step/read:0", shape=(), dtype=int64)

*** NNGenerator : layers= (2,3)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 1
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** persisted_tensors= CNN,<class 'numpy.ndarray'>,(),2_layer_CNN 

*** Iteration step= Tensor("adanet/iteration_1/step/read:0", shape=(), dtype=int64)
INFO:tensorflow:Building subnetwork '3_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_1/step/read:0", shape=(), dtype=int64)
INFO:tensorflow:Starting ensemble evaluation for iteration 1
INFO:tensorflow:Restoring parameters from ./tmp/adanet/CNN/model.ckpt-12
INFO:tensorflow:Encountered end of input after 440 evaluations
INFO:tensorflow:Computed ensemble metrics: adanet_loss/t0_2_layer_CNN 

INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_2_layer_CNN/weighted_subnetwork_1/subnetwork/dense_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_2_layer_CNN/weighted_subnetwork_1/subnetwork/dense_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_2_layer_CNN/weighted_subnetwork_1/logits/mixture_weight; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_2_layer_CNN/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/candidate_t1_2_layer_CNN/adanet_loss; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/candidate_t1_2_layer_CNN/adanet/iteration_1/candidate_t1_2_layer_CNN/adanet_loss/biased; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/candidate_t1_2_layer_CNN/adanet/iteration_1/candidate_t1_2_layer_CN

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18: ./tmp/adanet/CNN/model.ckpt-18
INFO:tensorflow:Loss for final step: 0.88700867.
INFO:tensorflow:Finished training Adanet iteration 2
INFO:tensorflow:Beginning bookkeeping phase for iteration 2
INFO:tensorflow:Importing architecture from ./tmp/adanet/CNN/architecture-1.txt: ['0:2_layer_CNN', '1:2_layer_CNN'].

*** NNGenerator : layers= (1,2)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 0
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_0/step/read:0", shape=(), dtype=int64)

*** NNGenerator : layers= (2,3)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 1
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** persisted_tensors= CNN,<class 'numpy.ndarray'>,(),2_layer_CNN 

***

...

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving candidate 't3_3_layer_CNN' dict for global step 30: accuracy/adanet/adanet_weighted_ensemble = 0.4680851, accuracy/adanet/subnetwork = 0.4893617, accuracy/adanet/uniform_average_ensemble = 0.4680851, architecture/adanet/ensembles = b'\n\x8b\x01\n>adanet/iteration_3/ensemble_t3_3_layer_CNN/architecture/adanetB?\x08\x07\x12\x00B9| 2_layer_CNN | 2_layer_CNN | 2_layer_CNN | 3_layer_CNN |J\x08\n\x06\n\x04text', average_loss/adanet/adanet_weighted_ensemble = 1.0327531, average_loss/adanet/subnetwork = 1.206189, average_loss/adanet/uniform_average_ensemble = 1.0327241, loss/adanet/adanet_weighted_ensemble = 1.0640161, loss/adanet/subnetwork = 1.1961517, loss/adanet/uniform_average_ensemble = 1.063976
INFO:tensorflow:Saving candidate 't4_3_layer_CNN' dict for global step 30: accuracy/adanet/adanet_weighted_ensemble = 0.5319149, accuracy/adanet/subnetwork = 0.3617021, accuracy/adanet/unifor

INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/batch_normalization/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/batch_normalization_1/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/batch_normalization_1/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_2_layer_CNN/weighted_subnetwork_0/subnetwork/batch_normalization_1/moving_mean; prev_var_name: Unchanged

INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/batch_normalization_1/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/batch_normalization_1/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/batch_normalization_1/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/batch_normalization_1/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_2/ensemble_t2_2_layer_CNN/weighted_subnetwork_2/subnetwork/dense_1/kernel; prev_var_name: Uncha

INFO:tensorflow:Finished bookkeeping phase for iteration 4
INFO:tensorflow:Beginning training AdaNet iteration 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Importing architecture from ./tmp/adanet/CNN/architecture-4.txt: ['0:2_layer_CNN', '1:2_layer_CNN', '2:2_layer_CNN', '3:3_layer_CNN'].

*** NNGenerator : layers= (1,2)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 0
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** Iteration step= Tensor("adanet/iteration_0/step/read:0", shape=(), dtype=int64)

*** NNGenerator : layers= (2,3)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Rebuilding iteration 1
INFO:tensorflow:Building subnetwork '2_layer_CNN'


*** build_subnetwork() : NN type= CNN

*** persisted_tensors= CNN,<class 'numpy.ndarray'>,(),2_layer_CNN 

*** Iteration step= Tensor("adanet/iteration_1/step/read:0", shape=(), dtype=int64)


INFO:tensorflow:Loss for final step: 0.88436675.
INFO:tensorflow:Finished training Adanet iteration 5


In [9]:
int('2_layer_DNN'.split('_')[0])

2

In [10]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("\n{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 346.9677803516388

ACCURACY: 0.5319148898124695

ACCURACY/ADANET/ADANET_WEIGHTED_ENSEMBLE: 0.5319148898124695

ACCURACY/ADANET/SUBNETWORK: 0.3617021143436432

ACCURACY/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 0.5319148898124695

ARCHITECTURE/ADANET/ENSEMBLES: b'\n\x99\x01\n>adanet/iteration_5/ensemble_t5_3_layer_CNN/architecture/adanetBM\x08\x07\x12\x00BG| 2_layer_CNN | 2_layer_CNN | 2_layer_CNN | 3_layer_CNN | 3_layer_CNN |J\x08\n\x06\n\x04text'

AVERAGE_LOSS: 1.0410116910934448

AVERAGE_LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.0410116910934448

AVERAGE_LOSS/ADANET/SUBNETWORK: 1.1128088235855103

AVERAGE_LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.0410197973251343

LOSS: 1.0609149932861328

LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.0609149932861328

LOSS/ADANET/SUBNETWORK: 1.0847151279449463

LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.0609111785888672

GLOBAL_STEP: 36


In [11]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [12]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [13]:
print("Architecture: {} / Accuracy= {} /Loss= {}".format(ensemble_architecture(results)\
                                                  , results['accuracy'], results['loss']))

Architecture: b'| 2_layer_CNN | 2_layer_CNN | 2_layer_CNN | 3_layer_CNN | 3_layer_CNN |' / Accuracy= 0.5319148898124695 /Loss= 1.0609149932861328


In [14]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNNBase_layer_1 |' / Accuracy= 0.40425533056259155 /Loss= 1.081491470336914
# Architecture: b'| CNNBase_linear | CNNBase_layer_1 |' / Accuracy= 0.42553192377090454 /Loss= 1.1250361204147339

In [15]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [16]:
[(i,i%2) for i in range(0,10)]

[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1)]

In [17]:
1%2

1

In [18]:
2%2

0

In [19]:
3%2

1

In [20]:
4%2

0